# Yacht Insurance Claims Data

Modeling to eventually do- randomforest,
<br>
Oversample on only the training data to prevent data leakage/bleeding? https://beckernick.github.io/oversampling-modeling/

**Problem Statement:** Using a regression model, our goal is to predict the probability that a yacht insurance policy holder is likley to have a loss within 5 years of signing up. Using this information we will aassign values to each characteristic of the boat and the customer themselves, in order to provide the ideal premium they should be charged.

**Audience:** President of Novamar Insurance Group 

In [1]:
import pandas as pd

In [2]:
claims = pd.read_excel('Claims_report.xlsx')

In [3]:
claims.head()

,Claim No.,Status,Insured name,Policy No.,Policy Eff-Date,Date of Ocurrence,Year,Length,Hull Value,Horsepower,...,Mooring State,Mooring County,Construction,Navigation Limits,Loss,Expense,Loss.1,Expense.1,To Date,Total Incurred
0,1.54715.MA.107,Open,Brent Cox,NYPCAF001878-2,2020-04-04,2020-11-03,1995.0,54.0,580000.0,1640.0,...,AL,BALDWIN,Fiberglass,(D) U.S. Atlantic and Gulf - 1,1000.00,500.0,0.00,0.00,0.0,1500.00
1,1.54181.107.MA,Closed,Brent Cox,NYPCAF001878-2,2020-04-04,2020-10-15,1995.0,54.0,580000.0,1640.0,...,AL,BALDWIN,Fiberglass,(D) U.S. Atlantic and Gulf - 1,618.84,500.0,0.00,0.00,0.0,1118.84
2,1.55475,Open,KEN & HEATHER DEVRIES,NYPCAF001239-3,2020-06-17,2020-10-01,1985.0,44.0,125000.0,60.0,...,NAYARIT,MEXICO,Fiberglass,(A) Pacific - 2,NaN,NaN,NaN,NaN,NaN,NaN
3,1.52983.107MA,Open,STEPHEN & BETH LEATHERS,NYPCAH001805-2,2020-03-05,2020-09-16,2004.0,42.0,300000.0,60.0,...,FL,ESCAMBIA,Fiberglass,(D) U.S. Atlantic and Gulf - 1,0.00,14000.0,310960.00,0.00,0.0,324960.00
4,1.53180.107.MA,Open,Ed Gassman,NYPCAF002704,2019-11-05,2020-08-22,2001.0,45.0,200000.0,900.0,...,FL,BROWARD,Fiberglass,(D) U.S. Atlantic and Gulf - 3,0.00,0.0,25481.25,25481.25,0.0,50962.50


In [4]:
claims.shape

(410, 25)

In [5]:
claims['Policy No.'].nunique()

374

In [6]:
claims['Claim No.'].nunique()

409

In [7]:
master = pd.read_csv('NYP Bordereau Report - Total.xlsx - Bordereau Report.csv')
master.head()

,Policy Number,New/Renl/Endt/Canc/Flat,Insurance Broker,First Name,Last Name,Loss Paye,Date of Birth,Married yes/no,Years Exp.,Occupation,...,Hull Type,Hull Limit,Power Type,# Engines,Mooring State,Mooring County,Mooring ZIP code,Credits,Debits,Windstorm Deductible
0,NYPCAF000314,New,Novamar Insurance,Richard,Spindler,Named Insured,05/09/1948,No,2.0,PUBLISHER,...,Multihull Sail,"$ 500,000.00",Inboard,2.0,Nayarit,Mexico,NaN,0%,35%,5%
1,NYPCAF000315,New,Novamar Insurance,Hoc,Tran,United Bank,10/08/1952,NaN,22.0,OWNS SCRAP METAL BUSINESS,...,Sportfisher,"$ 1,275,000.00",Inboard,2.0,CA,Los Angeles,90803,0%,20%,2%
2,NYPCAF000316,Flat,Novamar Insurance,KARL,KRUG,NaN,06/25/1956,Yes,24.0,CEO,...,Trawler,"$ 663,000.00",Inboard,2.0,WASHINGTON,King,98109,0%,0%,2%
3,NYPCAF000316,Flat,Novamar Insurance,KARL,KRUG,NaN,06/25/1956,Yes,24.0,CEO,...,Trawler,"$ 663,000.00",Inboard,2.0,WASHINGTON,King,98109,0%,0%,2%
4,NYPCAF000318,New,Novamar Insurance,JERRY,BORISY,NAMED INSURED,09/30/1943,Yes,30.0,-,...,Motoryacht,"$ 400,000.00",Inboard,2.0,CA,LOS ANGELES,90803,6%,0%,1%


In [8]:
master.shape

(7952, 45)

In [9]:
master['Policy Number'].nunique()

5553

In [10]:
claims['Policy No.'].nunique() / master['Policy Number'].nunique()

0.06735098145146767

### In 5 years, NovaMar could expect that 6.7% of all policies will have a claim based on historical data. Each year there are new policies and dropped policies, but overall you can expect there to be roughly 7% of all policies to have claims. Will need to oversample the target data for the model. 

In [11]:
master.columns

Index(['Policy Number', 'New/Renl/Endt/Canc/Flat', 'Insurance Broker',
       'First Name', 'Last Name', 'Loss Paye', 'Date of Birth',
       'Married yes/no', 'Years Exp.', 'Occupation', 'Mailing State',
       'Policy Eff-Date', 'Endo Eff-Date', 'Policy/Endo Canc Eff-Date',
       'Hull Prem', 'Primary Lia Prem', 'Excess Lia Prem', 'Primary Crew Prem',
       'Excess Crew Prem', 'Pol Fee', 'Total Prem Incl Fee',
       'Total Prem Less Fee', 'Collected Premium', 'Collected Fee',
       'Date Paid', 'Collected Total', 'Unpaid Premium', 'Unpaid Fee',
       'Unpaid Total', 'Net Prem Due RFIB', 'Vessel Name', 'Builder',
       'Year Built', 'Construction', 'Length', 'Hull Type', 'Hull Limit',
       'Power Type', '# Engines', 'Mooring State', 'Mooring County',
       'Mooring ZIP code', 'Credits', 'Debits', 'Windstorm Deductible'],
      dtype='object')

In [12]:
claims.columns

Index(['Claim No.', 'Status', 'Insured name', 'Policy No.', 'Policy Eff-Date',
       'Date of Ocurrence', 'Year', 'Length', 'Hull Value', 'Horsepower',
       'Accident description', 'Manufacturer', 'Hull Type', 'Power Type',
       'Fuel Type', 'Mooring State', 'Mooring County', 'Construction',
       'Navigation Limits', 'Loss', 'Expense', 'Loss.1', 'Expense.1',
       'To Date', 'Total Incurred'],
      dtype='object')

---

# Data Cleaning

**BELOW:** Check to see if columns to merge on (policy number) seem likely to match up.

In [13]:
# Number of total policy numbers in the claims data
len(claims['Policy No.'])

410

In [14]:
# Number of unique policy numbers in the claims data
claims['Policy No.'].nunique()

374

In [15]:
# Number with repeat claims
len(claims['Policy No.']) - claims['Policy No.'].nunique()

36

In [16]:
claims['Policy No.'].head(10)

0    NYPCAF001878-2
1    NYPCAF001878-2
2    NYPCAF001239-3
3    NYPCAH001805-2
4      NYPCAF002704
5    NYPCAF002449-1
6    NYPCAF000448-3
7      NYPCAF002767
8      NYPCAF002689
9    NYPCAF002337-1
Name: Policy No., dtype: object

In [17]:
master['Policy Number'].tail(10)

7942    NYPCAF001953-2
7943      NYPCAF002700
7944    NYPCAH000986-3
7945      NYPCAF002762
7946    NYPCAF002391-1
7947    NYPCAF002391-1
7948    NYPCAH001020-3
7949    NYPCAH001020-3
7950    NYPCAF000564-2
7951               NaN
Name: Policy Number, dtype: object

In [18]:
print(claims['Policy No.'].isna().sum())
print(master['Policy Number'].isna().sum())

1
1


In [19]:
# Drop nulls in both datasets for policy numbers
claims.dropna(subset = ['Policy No.'],inplace=True)
master.dropna(subset = ['Policy Number'], inplace= True)

In [20]:
print(claims['Policy No.'].isna().sum())
print(master['Policy Number'].isna().sum())

0
0


In [21]:
master_policy = master['Policy Number'].to_list()
claims_policy = claims['Policy No.'].to_list()

In [22]:
common_policy = [value for value in master_policy if value in claims_policy]
len(common_policy)

490

**Below:** This means all of the claims with unique policy numbers from the claims database have matching policy numbers in the master database.

In [23]:
len(set(common_policy))

374

### Take a look at why there are repeat policies in the master dataframe.

In [24]:
master['Policy Number'][00:2010]

0         NYPCAF000314
1         NYPCAF000315
2         NYPCAF000316
3         NYPCAF000316
4         NYPCAF000318
             ...      
2005    NYPCAF000888-1
2006    NYPCAF000856-1
2007    NYPCAF001026-1
2008    NYPCAF000933-1
2009    NYPCAF000934-1
Name: Policy Number, Length: 2010, dtype: object

In [25]:
print(master.shape)
print(master['Policy Number'].nunique())

(7951, 45)
5553


In [26]:
master['New/Renl/Endt/Canc/Flat'].value_counts()

Renl         2298
New          2184
Endt         1207
Flat         1034
Endt-Flat     598
Canc          550
Endt-Canc      80
Name: New/Renl/Endt/Canc/Flat, dtype: int64

**BELOW:** Every entry labeled as "Canc" has the same number of Policy/Endo entries. Therefore, this is the length the policy lasted.

In [27]:
master.groupby('New/Renl/Endt/Canc/Flat')[['Policy Eff-Date','Policy/Endo Canc Eff-Date','Endo Eff-Date']].count()

,Policy Eff-Date,Policy/Endo Canc Eff-Date,Endo Eff-Date
New/Renl/Endt/Canc/Flat,,,
Canc,550,550,0
Endt,1207,14,1207
Endt-Canc,80,79,80
Endt-Flat,598,598,598
Flat,1034,1034,0
New,2184,16,0
Renl,2298,40,0


In [28]:
master.loc[master['New/Renl/Endt/Canc/Flat'] == 'Canc']

,Policy Number,New/Renl/Endt/Canc/Flat,Insurance Broker,First Name,Last Name,Loss Paye,Date of Birth,Married yes/no,Years Exp.,Occupation,...,Hull Type,Hull Limit,Power Type,# Engines,Mooring State,Mooring County,Mooring ZIP code,Credits,Debits,Windstorm Deductible
41,NYPCAF000355,Canc,Novamar Insurance,Gregory,Lynch,BANK OF AMERICA,01/04/1955,Yes,30.0,-,...,Motoryacht,"$ 625,000.00",Inboard,2.0,California,Orange,92661,0%,0%,2%
53,NYPCAF000368,Canc,Novamar Insurance,ROBERT T,BRINKLEY II,NaN,07/08/1945,Yes,40.0,YACHT BROKER,...,Motoryacht,"$ 135,000.00",Inboard,1.0,FL,Broward,33019,0%,6%,5%
60,NYPCAF000376,Canc,Novamar Insurance,Steve,Rowland,MB FINANCIAL BANK,08/22/1955,Yes,15.0,Retired,...,Motoryacht,"$ 59,950.00",Inboard,2.0,FL,BAY,32411,2%,40%,5%
63,NYPCAF000379,Canc,Novamar Insurance,BRETT,BASHAW,NaN,04/02/1963,Yes,30.0,COMPANY OWNER,...,Express Cruiser,"$ 700,000.00",Pod,2.0,CA,ORANGE,92661,4%,0%,1%
67,NYPCAF000383,Canc,Novamar Insurance,ARMAND,FAGGIONI,NaN,08/08/1958,Yes,25.0,RETIRED,...,Motoryacht,"$ 330,000.00",Inboard,2.0,FL,MANATEE,34221,0%,25%,5%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6018,NYPCAF001964-2,Canc,Total Dollar Yacht Insurance - FL,Reinaldo,Chavez,SPACE COAST CREDIT UNION,08/22/1962,Yes,38.0,Business Owner,...,Sportfisher,"$ 500,000.00",Inboard,2.0,FL,MIAMI-DADE,33141,0%,45%,5%
6023,NYPCAF001049-3,Canc,Novamar Insurance,Les,Bauer,BB&T - Insurance Center,12/13/1942,Yes,20.0,Retired,...,Motoryacht,"$ 1,450,000.00",Inboard,2.0,FL,PINELLAS,33701,0%,17%,5%
6038,NYPCAH001980-2,Canc,Novamar Insurance,Stephen,DePasquale,Pinnacle Bank,06/11/1965,Yes,10.0,Physician,...,Motoryacht,"$ 1,600,000.00",Inboard,2.0,TN,HAMILTON,37416,0%,36%,5%
6044,NYPCAF002575-1,Canc,Total Dollar Yacht Insurance - FL,Eduardo O.,Portero,NaN,04/04/1943,No,49.0,President of Company,...,Motoryacht,"$ 500,000.00",Inboard,2.0,FL,MIAMI DADE,33160,0%,50%,5%


Keep only one row per unique policy number? Choose the one with the most recent date?

In [29]:
master.shape

(7951, 45)

In [30]:
print(master['Policy Eff-Date'].count())
print(master['Endo Eff-Date'].count())
print(master['Policy/Endo Canc Eff-Date'].count())

7951
1885
2331


In [31]:
master.columns

Index(['Policy Number', 'New/Renl/Endt/Canc/Flat', 'Insurance Broker',
       'First Name', 'Last Name', 'Loss Paye', 'Date of Birth',
       'Married yes/no', 'Years Exp.', 'Occupation', 'Mailing State',
       'Policy Eff-Date', 'Endo Eff-Date', 'Policy/Endo Canc Eff-Date',
       'Hull Prem', 'Primary Lia Prem', 'Excess Lia Prem', 'Primary Crew Prem',
       'Excess Crew Prem', 'Pol Fee', 'Total Prem Incl Fee',
       'Total Prem Less Fee', 'Collected Premium', 'Collected Fee',
       'Date Paid', 'Collected Total', 'Unpaid Premium', 'Unpaid Fee',
       'Unpaid Total', 'Net Prem Due RFIB', 'Vessel Name', 'Builder',
       'Year Built', 'Construction', 'Length', 'Hull Type', 'Hull Limit',
       'Power Type', '# Engines', 'Mooring State', 'Mooring County',
       'Mooring ZIP code', 'Credits', 'Debits', 'Windstorm Deductible'],
      dtype='object')

In [32]:
print(master['Endo Eff-Date'].isna().sum())
print(master['Policy/Endo Canc Eff-Date'].isna().sum())

6066
5620


In [33]:
eed_no_na = (master['Endo Eff-Date'].dropna()).to_list()
peced_no_na = (master['Policy/Endo Canc Eff-Date'].dropna()).to_list()

In [34]:
len([value for value in eed_no_na if value not in peced_no_na])

387

In [35]:
len(master[master['Endo Eff-Date'] == master['Policy/Endo Canc Eff-Date']])

608

Trying to figure out which policy number to keep. Some have multiple rows and I want to keep only one them, the one with the most information (i.e. if it's been cancelled).

In [36]:
master[['Endo Eff-Date','Policy/Endo Canc Eff-Date']]

,Endo Eff-Date,Policy/Endo Canc Eff-Date
0,NaN,NaN
1,NaN,NaN
2,NaN,03/22/2019
3,NaN,03/22/2019
4,NaN,NaN
...,...,...
7946,08/19/2020,09/17/2020
7947,08/31/2020,09/17/2020
7948,09/27/2020,09/27/2020
7949,10/29/2020,10/29/2020


In [37]:
master.dtypes

Policy Number                 object
New/Renl/Endt/Canc/Flat       object
Insurance Broker              object
First Name                    object
Last Name                     object
Loss Paye                     object
Date of Birth                 object
Married yes/no                object
Years Exp.                   float64
Occupation                    object
Mailing State                 object
Policy Eff-Date               object
Endo Eff-Date                 object
Policy/Endo Canc Eff-Date     object
Hull Prem                     object
Primary Lia Prem              object
Excess Lia Prem               object
Primary Crew Prem             object
Excess Crew Prem              object
Pol Fee                       object
Total Prem Incl Fee           object
Total Prem Less Fee           object
Collected Premium            float64
Collected Fee                float64
Date Paid                    float64
Collected Total              float64
Unpaid Premium               float64
U

What to do about the repeat claims with the same policy number when merging? Should we combine them somehow? Is one test whether someone had one claim already? Don't want to combine them.

Would maybe want to create a duplicate row. Like, if this matches a policy number create a row for it.

### Turn dates into date/time columns

In [38]:
master[['Policy Eff-Date', 'Endo Eff-Date', ]].apply(pd.to_datetime)

,Policy Eff-Date,Endo Eff-Date
0,2016-03-31,NaT
1,2016-03-18,NaT
2,2019-03-22,NaT
3,2019-03-22,NaT
4,2016-03-30,NaT
...,...,...
7946,2020-03-15,2020-08-19
7947,2020-03-15,2020-08-31
7948,2020-05-01,2020-09-27
7949,2020-05-01,2020-10-29


**Below:** There was an error converting to date time because one DOB was set to 9605. The policy associated with this DOB was not in the claims report so I decided to drop it.

In [39]:
# Create a list of all the DOB years
year_list = []
for n in master['Date of Birth']:
    year_list.append(n[6:10])
year_list[:5]

['1948', '1952', '1956', '1956', '1943']

In [40]:
# Create DOB year column
master['DOB_year'] = year_list

# Check the biggest years to see if they make sense
master['DOB_year'].sort_values(ascending=False)

5030    9605
3261    9605
5734    9407
4115    9407
3406    2036
        ... 
1769    1932
5895    1931
4351    1931
2620    1931
2251    1928
Name: DOB_year, Length: 7951, dtype: object

In [41]:
master['DOB_year'] = pd.to_numeric(master['DOB_year'])

In [42]:
# How many incorrect birthdays?
print(len(master[master['DOB_year'] > 2000]))

# Create a dataframe to share these errors with the client
wrong_bdays = master[master['DOB_year'] > 2000]

# Export the dataframe to csv
wrong_bdays.to_csv('wrong_bdays.csv')

30


In [43]:
# Drop all rows where DOB year is greater than 2000
master.drop(master.loc[master['DOB_year'] > 2000].index, inplace=True)

In [44]:
master[master['DOB_year'] > 2000]

,Policy Number,New/Renl/Endt/Canc/Flat,Insurance Broker,First Name,Last Name,Loss Paye,Date of Birth,Married yes/no,Years Exp.,Occupation,...,Hull Limit,Power Type,# Engines,Mooring State,Mooring County,Mooring ZIP code,Credits,Debits,Windstorm Deductible,DOB_year


In [45]:
# Create and age column
master['age'] = 2020 - master['DOB_year']
master['age'].sort_values

<bound method Series.sort_values of 0       72
1       68
2       64
3       64
4       77
        ..
7944    62
7945    53
7946    56
7947    56
7950    62
Name: age, Length: 7921, dtype: int64>

In [46]:
master['Policy/Endo Canc Eff-Date'] = master['Policy/Endo Canc Eff-Date'].astype(str)

In [47]:
# Create a list of all the Canc date years
years_list = []
for n in master['Policy/Endo Canc Eff-Date']:
    years_list.append(n[6:10])
years_list[:10]

['', '', '2019', '2019', '', '', '', '', '', '']

In [48]:
# Look at the dates to see where the error is
sorted(years_list, reverse=True)[:10]

['9999',
 '9999',
 '9999',
 '9999',
 '9000',
 '9000',
 '9000',
 '9000',
 '9000',
 '9000']

In [49]:
# Create column of year cancelled
master['canc_year'] = years_list

# Turn it into a nmeric column
master['canc_year']  = pd.to_numeric(master['canc_year'])

# Look at how many problems there are
len(master[master['canc_year'] > 2020])

10

In [50]:
# Drop rows with dates over 2020
master.drop(master.loc[master['canc_year'] > 2020].index, inplace=True)

In [59]:
# Now turn endorsement/policy cancellation date into date/time object
pd.to_datetime(master['Policy/Endo Canc Eff-Date'])

0             NaT
1             NaT
2      2019-03-22
3      2019-03-22
4             NaT
          ...    
7944   2020-04-17
7945   2020-08-15
7946   2020-09-17
7947   2020-09-17
7950   2020-11-04
Name: Policy/Endo Canc Eff-Date, Length: 7911, dtype: datetime64[ns]

In [58]:
master.dtypes

Policy Number                 object
New/Renl/Endt/Canc/Flat       object
Insurance Broker              object
First Name                    object
Last Name                     object
Loss Paye                     object
Date of Birth                 object
Married yes/no                object
Years Exp.                   float64
Occupation                    object
Mailing State                 object
Policy Eff-Date               object
Endo Eff-Date                 object
Policy/Endo Canc Eff-Date     object
Hull Prem                     object
Primary Lia Prem              object
Excess Lia Prem               object
Primary Crew Prem             object
Excess Crew Prem              object
Pol Fee                       object
Total Prem Incl Fee           object
Total Prem Less Fee           object
Collected Premium            float64
Collected Fee                float64
Date Paid                    float64
Collected Total              float64
Unpaid Premium               float64
U

In [88]:
# Create length of policy held column
master['policy_length'] = master['Policy/Endo Canc Eff-Date'] - master['Policy Eff-Date']
master.groupby('New/Renl/Endt/Canc/Flat')['policy_length'].mean()

TypeError: cannot subtract DatetimeArray from ndarray

Questions:<br>
- revisit up above  column, some values may not make sense
- are all endt-flat repeat policies that exist in the other categories? should i just drop them all?
- should i get rid of flat one all together?
- why aren't these now converting to date/time objects?